<a href="https://colab.research.google.com/github/natfontanesi/Case_Ifood/blob/main/Case_Ifood.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importações

In [1]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install spotipy

     |████████████████████████████████| 62 kB 1.0 MB/s 
     |████████████████████████████████| 138 kB 17.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.26.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [3]:

from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Variáveis e funções para todo o notebook

In [4]:
month = {	
    1:'January',
		2:'February',
		3:'March',
		4:'April',
		5:'May',
		6:'June',
		7:'July',
		8:'August',
		9:'September',
		10:'October',
		11:'November',
		12:'December'		
}

In [5]:
def get_part_of_day(x):
    if (x > 4) and (x <= 8):
        return 'Early Morning'
    elif (x > 8) and (x <= 12 ):
        return 'Morning'
    elif (x > 12) and (x <= 16):
        return'Noon'
    elif (x > 16) and (x <= 20) :
        return 'Evening'
    elif (x > 20) and (x <= 24):
        return'Night'
    elif (x <= 4):
        return'Late Night'

In [6]:
def clean_title(text):
  if len(text.split(':')) <= 3:
    return ' '.join(text.split(':')[:1])
  else:
    return ' '.join(text.split(':')[:2])


# <font color= red> Ifood

## Carregando o arquivo json

In [7]:
ifood = pd.read_json('/content/drive/MyDrive/3_VAMO_AI/01_Case_Ifood/ifood-vamoai.json', encoding='UTF-8')
ifood.head()

,id,shortId,createdAt,updatedAt,closedAt,lastStatus,details,delivery,merchant,payments,bag,origin,deliveryMethod,review
0,5f2c646e-2e52-497f-8fa3-5d642f43ce45,2919873206,2021-02-25T12:22:01-03:00,2021-02-25T12:40:32-03:00,2021-02-25T12:39:21-03:00,CONCLUDED,"{'mode': 'DELIVERY', 'scheduled': False, 'tipp...","{'address': {'city': 'Ferraz de Vasconcelos', ...","{'address': {'establishment': '', 'city': 'POA...",{'methods': [{'id': '83e91c55-f778-430f-a13a-0...,"{'benefits': [], 'deliveryFee': {'value': 549,...","{'platform': 'ANDROID', 'appName': 'iFood', 'a...","{'id': 'DEFAULT', 'mode': 'DELIVERY'}",NaN
1,366b27ed-9e4d-4da5-bbb5-e21ded9b3520,2900401480,2021-02-19T19:39:01-03:00,2021-02-19T20:03:37-03:00,2021-02-19T20:03:05-03:00,CONCLUDED,"{'mode': 'DELIVERY', 'scheduled': False, 'tipp...","{'address': {'city': 'Ferraz de Vasconcelos', ...","{'address': {'establishment': '', 'city': 'POA...",{'methods': [{'id': '13e5507f-c1da-45b4-b8b8-5...,"{'benefits': [], 'deliveryFee': {'value': 649,...","{'platform': 'ANDROID', 'appName': 'iFood', 'a...","{'id': 'DEFAULT', 'mode': 'DELIVERY'}",NaN
2,16ea5252-dba3-49d9-99ca-d39f6266c367,2886987691,2021-02-16T12:35:43-03:00,2021-02-16T13:05:45-03:00,2021-02-16T13:04:42-03:00,CONCLUDED,"{'mode': 'DELIVERY', 'scheduled': False, 'tipp...","{'address': {'city': 'Ferraz de Vasconcelos', ...","{'address': {'establishment': '', 'city': 'POA...",{'methods': [{'id': '13e5507f-c1da-45b4-b8b8-5...,"{'benefits': [], 'deliveryFee': {'value': 549,...","{'platform': 'ANDROID', 'appName': 'iFood', 'a...","{'id': 'DEFAULT', 'mode': 'DELIVERY'}",NaN
3,aef66019-39db-457b-8c95-3ece0a25e9e6,2874260444,2021-02-12T12:33:51-03:00,2021-02-12T13:13:12-03:00,2021-02-12T13:11:18-03:00,CONCLUDED,"{'mode': 'DELIVERY', 'scheduled': False, 'tipp...","{'address': {'city': 'Ferraz de Vasconcelos', ...","{'address': {'establishment': '', 'city': 'POA...",{'methods': [{'id': '13e5507f-c1da-45b4-b8b8-5...,"{'benefits': [], 'deliveryFee': {'value': 549,...","{'platform': 'ANDROID', 'appName': 'iFood', 'a...","{'id': 'DEFAULT', 'mode': 'DELIVERY'}",NaN
4,5a7d800a-c559-4b05-8ea1-202bfa6d012a,2870658072,2021-02-11T17:36:30-03:00,2021-02-11T18:06:33-03:00,2021-02-11T18:04:51-03:00,CONCLUDED,"{'mode': 'DELIVERY', 'scheduled': False, 'tipp...","{'address': {'city': 'Ferraz de Vasconcelos', ...","{'address': {'establishment': '', 'city': 'POA...",{'methods': [{'id': '13e5507f-c1da-45b4-b8b8-5...,"{'benefits': [], 'deliveryFee': {'value': 649,...","{'platform': 'ANDROID', 'appName': 'iFood', 'a...","{'id': 'DEFAULT', 'mode': 'DELIVERY'}","{'reviewedAt': '2021-02-11T18:06:33-03:00', 's..."


## Transformando as colunas aninhadas em novos dataframes

In [8]:
details=pd.json_normalize(ifood['details'])
delivery=pd.json_normalize(ifood['delivery'])
merchant=pd.json_normalize(ifood['merchant'])
payments=pd.json_normalize(ifood['payments'])
origin=pd.json_normalize(ifood['origin'])
bag=pd.json_normalize(ifood['bag'])


In [9]:
ifood.head(1)

,id,shortId,createdAt,updatedAt,closedAt,lastStatus,details,delivery,merchant,payments,bag,origin,deliveryMethod,review
0,5f2c646e-2e52-497f-8fa3-5d642f43ce45,2919873206,2021-02-25T12:22:01-03:00,2021-02-25T12:40:32-03:00,2021-02-25T12:39:21-03:00,CONCLUDED,"{'mode': 'DELIVERY', 'scheduled': False, 'tipp...","{'address': {'city': 'Ferraz de Vasconcelos', ...","{'address': {'establishment': '', 'city': 'POA...",{'methods': [{'id': '83e91c55-f778-430f-a13a-0...,"{'benefits': [], 'deliveryFee': {'value': 549,...","{'platform': 'ANDROID', 'appName': 'iFood', 'a...","{'id': 'DEFAULT', 'mode': 'DELIVERY'}",NaN


## Vendo os tamanhos de cada df novo

In [10]:
bag.shape

(82, 9)

In [11]:
origin.shape

(82, 3)

In [12]:
payments.shape

(82, 3)

In [13]:
merchant.shape

(82, 17)

In [14]:
delivery.shape

(82, 18)

In [15]:
details.shape

(82, 11)

In [16]:
ifood.shape

(82, 14)

## Desaninhando alguns DFs

In [17]:
payments.head(1)

,methods,total.currency,total.value
0,[{'id': '83e91c55-f778-430f-a13a-0467508fd9df'...,BRL,2549


In [18]:
payments=payments.explode('methods')

In [19]:
payments_methods=pd.json_normalize(payments['methods'])

In [20]:
payments_methods.head(1)

,id,transactions,method.name,method.description,type.name,type.description,brand.id,brand.image,brand.name,brand.description,mealVoucher.cardNumber,amount.currency,amount.value,credit.cardNumber,digitalWallet.id,digitalWallet.name,digitalWallet.description,digitalWallet.image,digitalWallet.cardNumber,digitalWallet.usedBalance.currency,digitalWallet.usedBalance.value,debit.cardNumber
0,83e91c55-f778-430f-a13a-0467508fd9df,"[{'id': '64027405615-640274-004836', 'type': '...",MEAL_VOUCHER,Vale-refeição,ONLINE,Pago pelo app,07b1c8c2-0e38-424e-9fa2-6ea70e5f1dd3,07b1c8c2-0e38-424e-9fa2-6ea70e5f1dd3.png,VR,VR refeição,****************,BRL,2549,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
payments_methods.shape

(82, 22)

In [22]:
bag.head(1)

,benefits,items,updated,deliveryFee.value,deliveryFee.valueWithDiscount,subTotal.value,subTotal.valueWithDiscount,total.value,total.valueWithDiscount
0,[],[{'id': 'f7e42041-6b2c-4b33-94ac-464117949826'...,False,549,549,2000,2000,2549,2549


In [23]:
bag[['deliveryFee.value','total.valueWithDiscount']]=bag[['deliveryFee.value','total.valueWithDiscount']].div(100)

In [24]:
bag=bag.explode('items')

In [25]:
bag_itens=pd.json_normalize(bag['items'])

In [26]:
bag_itens

,id,uniqueId,externalId,name,description,quantity,subItems,tags,totalPrice,totalPriceWithDiscount,unitPrice,unitPriceWithDiscount,notes
0,f7e42041-6b2c-4b33-94ac-464117949826,ceb42ac6-5983-4d45-89ee-031c9babeb3f,267804,Picadinho,"Arroz, feijão, picadinho e salada.\n+ (1 Bola ...",1,[{'id': '19ada6bc-aa18-4a6e-8f6e-7b3e059e5db9'...,"[TAG::DIADOCLIENTE_ACAI_DOCES_15, TAG::ENTREGA...",2000,2000,1500,1500,NaN
1,a67f0608-2a70-4ec8-bcb8-75984a14bd7c,417b16ca-6d9e-497f-9c68-8976bea3840e,529880001,2 Sanduíches + Batata + Bebida,2 sanduíches (Whopper ou Cheeseburger Duplo B...,3,[{'id': '57632132-e582-4363-b74a-c2f0f8c363ad'...,[],11670,11670,3890,3890,NaN
2,df6ea899-8f57-468a-ba04-9cd4711998da,6b5bd717-c787-429f-8d72-22f745340deb,264996,Parmegiana de frango ou carne,"Arroz, feijão, fritas e salada.\n+(1 Bola de a...",2,[{'id': '277668c4-265e-4530-981b-d5ebc387098e'...,"[TAG::ENTREGA_FOGUETE_NOV19, TAG::MARMITA_FORA...",4000,4000,2000,2000,NaN
3,f7e42041-6b2c-4b33-94ac-464117949826,3502ffc6-e850-4306-921e-97f3ee3b52e3,267804,Picadinho,"Arroz, feijão, picadinho e salada.\n+ (1 Bola ...",1,[{'id': '19ada6bc-aa18-4a6e-8f6e-7b3e059e5db9'...,"[TAG::DIADOCLIENTE_ACAI_DOCES_15, TAG::ENTREGA...",2000,2000,1500,1500,NaN
4,fa51badc-23d6-45a0-91f9-2dc17b04aa19,5f3eba1f-e07f-4dd7-8978-3e6a7319115e,267561,Filé de peixe,"Filé de peixe com batata fritas, arroz, feijão...",1,[{'id': '5dcce42d-5d2a-45b8-a259-5e6018bd8b5f'...,"[TAG::ENTREGA_FOGUETE_NOV19, TAG::MARMITA_FORA...",1800,1800,1800,1800,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,NaN,NaN,70613595,Talharim à bolonhesa 700 gramas,NaN,1,"[{'externalId': '70613624', 'name': 'Não desej...",[TAG::BEST_SELLERS],2699,2699,2699,2699,NaN
143,NaN,NaN,70610541,Parmegiana de frango 700 gramas,NaN,1,"[{'externalId': '70610571', 'name': 'Feijão ca...","[TAG::CRM_MULTI_CAT_BRA, TAG::BEST_SELLERS]",3398,3398,2699,2699,NaN
144,NaN,NaN,9,Linguiça toscana a brasileira,NaN,1,[],"[TAG::ALMOCO_ATE_15, TAG::HUB_2611, TAG::HUB_G...",1099,1099,1099,1099,NaN
145,NaN,NaN,9,Linguiça toscana a brasileira,NaN,1,[],"[TAG::ALMOCO_ATE_15, TAG::HUB_2611, TAG::HUB_G...",990,990,990,990,NaN


In [27]:
bag.shape

(147, 9)

In [28]:
bag_itens.shape

(147, 13)

In [29]:
details.head(1)

,mode,scheduled,tippable,indoorTipEnabled,trackable,boxable,placedAtBox,reviewed,darkKitchen,cancellation.code,cancellation.reason
0,DELIVERY,False,True,False,True,False,False,False,False,NaN,NaN


In [30]:
merchant.head(1)

,id,name,phoneNumber,logo,companyGroup,type,address.establishment,address.city,address.country,address.neighborhood,address.state,address.streetName,address.streetNumber,address.coordinates.latitude,address.coordinates.longitude,address.reference,address.complement
0,84d619e6-9c4c-44a7-83c3-e52632f80e52,Spumoni Restaurante e Sorveteria,40030053,84d619e6-9c4c-44a7-83c3-e52632f80e52/201909261...,IFD,RESTAURANT,,POA,BR,VILA ACOREANA,SP,R RAIMUNDO CORREIA,38,-23.517951,-46.341367,,


In [31]:
merchant.rename(columns={'address.city': 'r.adress.city','address.country':'r.address.country','address.neighborhood':'r.address.neighborhood','address.state':'r.address.state','address.streetName':'r.address.streetName','address.streetNumber':'r.address.streetNumber','address.coordinates.latitude':'r.address.coordinates.latitude','address.coordinates.longitude':'r.address.coordinates.longitude','address.reference':'r.address.reference','address.complement':'r.address.complement' },inplace=True)

In [32]:
delivery.head(1)

,expectedDeliveryTime,expectedDuration,address.city,address.country,address.neighborhood,address.state,address.streetName,address.streetNumber,address.coordinates.latitude,address.coordinates.longitude,address.complement,driver.id,driver.name,driver.photoUrl,driver.modal,estimatedTimeOfArrival.deliversAt,estimatedTimeOfArrival.updatedAt,estimatedTimeOfArrival.reason
0,2021-02-25T12:43:01-03:00,1260,Ferraz de Vasconcelos,BR,Vila Andeyara,SP,Praça da Biblia,44,-23.540557,-46.369043,,050f3047-2878-4a80-b56b-5a0bb3c349b9,Gildarle Silva,https://nv-production-logistics-driver-account...,MOTORCYCLE,2021-02-25T12:43:01-03:00,2021-02-25T12:40:32-03:00,NaN


In [33]:
delivery.rename({'adress.city':'user.adress.city'},axis='index',inplace=True)

In [34]:
delivery.head(1)

,expectedDeliveryTime,expectedDuration,address.city,address.country,address.neighborhood,address.state,address.streetName,address.streetNumber,address.coordinates.latitude,address.coordinates.longitude,address.complement,driver.id,driver.name,driver.photoUrl,driver.modal,estimatedTimeOfArrival.deliversAt,estimatedTimeOfArrival.updatedAt,estimatedTimeOfArrival.reason
0,2021-02-25T12:43:01-03:00,1260,Ferraz de Vasconcelos,BR,Vila Andeyara,SP,Praça da Biblia,44,-23.540557,-46.369043,,050f3047-2878-4a80-b56b-5a0bb3c349b9,Gildarle Silva,https://nv-production-logistics-driver-account...,MOTORCYCLE,2021-02-25T12:43:01-03:00,2021-02-25T12:40:32-03:00,NaN


dropando as colunas aninhadas para concatenar os novos df

In [35]:
ifood.drop(labels=['details','delivery','merchant','payments','origin'],axis=1, inplace=True)

## DF ifood

In [36]:
ifood.shape

(82, 9)

convertendo  em datetime para separar o ano, mes, dia e dia da semana

In [37]:
ifood['createdAt']=pd.to_datetime(ifood['createdAt'])

In [38]:
ifood['ano']=pd.DatetimeIndex(ifood['createdAt']).year
ifood['mes']=pd.DatetimeIndex(ifood['createdAt']).month
ifood['dia']=pd.DatetimeIndex(ifood['createdAt']).day
ifood['diaDaSemana']=ifood['createdAt'].dt.day_name()
ifood.head(1)

,id,shortId,createdAt,updatedAt,closedAt,lastStatus,bag,deliveryMethod,review,ano,mes,dia,diaDaSemana
0,5f2c646e-2e52-497f-8fa3-5d642f43ce45,2919873206,2021-02-25 12:22:01-03:00,2021-02-25T12:40:32-03:00,2021-02-25T12:39:21-03:00,CONCLUDED,"{'benefits': [], 'deliveryFee': {'value': 549,...","{'id': 'DEFAULT', 'mode': 'DELIVERY'}",NaN,2021,2,25,Thursday


In [39]:
ifood.mes.replace(month,inplace=True)

concatenando os novos dataframes em um só

In [40]:
lista= [ifood,details,delivery,merchant,payments,origin,payments_methods]
ifood=pd.concat(lista,axis=1)

In [41]:
ifood.shape

(82, 87)

In [42]:
ifood.head(1)

,id,shortId,createdAt,updatedAt,closedAt,lastStatus,bag,deliveryMethod,review,ano,mes,dia,diaDaSemana,mode,scheduled,tippable,indoorTipEnabled,trackable,boxable,placedAtBox,reviewed,darkKitchen,cancellation.code,cancellation.reason,expectedDeliveryTime,expectedDuration,address.city,address.country,address.neighborhood,address.state,address.streetName,address.streetNumber,address.coordinates.latitude,address.coordinates.longitude,address.complement,driver.id,driver.name,driver.photoUrl,driver.modal,estimatedTimeOfArrival.deliversAt,...,type,address.establishment,r.adress.city,r.address.country,r.address.neighborhood,r.address.state,r.address.streetName,r.address.streetNumber,r.address.coordinates.latitude,r.address.coordinates.longitude,r.address.reference,r.address.complement,methods,total.currency,total.value,platform,appName,appVersion,id,transactions,method.name,method.description,type.name,type.description,brand.id,brand.image,brand.name,brand.description,mealVoucher.cardNumber,amount.currency,amount.value,credit.cardNumber,digitalWallet.id,digitalWallet.name,digitalWallet.description,digitalWallet.image,digitalWallet.cardNumber,digitalWallet.usedBalance.currency,digitalWallet.usedBalance.value,debit.cardNumber
0,5f2c646e-2e52-497f-8fa3-5d642f43ce45,2919873206,2021-02-25 12:22:01-03:00,2021-02-25T12:40:32-03:00,2021-02-25T12:39:21-03:00,CONCLUDED,"{'benefits': [], 'deliveryFee': {'value': 549,...","{'id': 'DEFAULT', 'mode': 'DELIVERY'}",NaN,2021,February,25,Thursday,DELIVERY,False,True,False,True,False,False,False,False,NaN,NaN,2021-02-25T12:43:01-03:00,1260,Ferraz de Vasconcelos,BR,Vila Andeyara,SP,Praça da Biblia,44,-23.540557,-46.369043,,050f3047-2878-4a80-b56b-5a0bb3c349b9,Gildarle Silva,https://nv-production-logistics-driver-account...,MOTORCYCLE,2021-02-25T12:43:01-03:00,...,RESTAURANT,,POA,BR,VILA ACOREANA,SP,R RAIMUNDO CORREIA,38,-23.517951,-46.341367,,,"{'id': '83e91c55-f778-430f-a13a-0467508fd9df',...",BRL,2549,ANDROID,iFood,9.90.0,83e91c55-f778-430f-a13a-0467508fd9df,"[{'id': '64027405615-640274-004836', 'type': '...",MEAL_VOUCHER,Vale-refeição,ONLINE,Pago pelo app,07b1c8c2-0e38-424e-9fa2-6ea70e5f1dd3,07b1c8c2-0e38-424e-9fa2-6ea70e5f1dd3.png,VR,VR refeição,****************,BRL,2549,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
ifood.drop(labels=['deliveryMethod','digitalWallet.cardNumber','digitalWallet.description','digitalWallet.id','digitalWallet.image','digitalWallet.name','digitalWallet.usedBalance.currency','digitalWallet.usedBalance.value','mealVoucher.cardNumber','transactions'],axis=1, inplace=True)

In [44]:
ifood=ifood.infer_objects()

In [45]:
ifood.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82 entries, 0 to 81
Data columns (total 77 columns):
 #   Column                             Non-Null Count  Dtype                                 
---  ------                             --------------  -----                                 
 0   id                                 82 non-null     object                                
 1   shortId                            82 non-null     int64                                 
 2   createdAt                          82 non-null     datetime64[ns, pytz.FixedOffset(-180)]
 3   updatedAt                          82 non-null     object                                
 4   closedAt                           52 non-null     object                                
 5   lastStatus                         82 non-null     object                                
 6   bag                                82 non-null     object                                
 7   review                             6 

In [46]:
ifood.columns

Index(['id', 'shortId', 'createdAt', 'updatedAt', 'closedAt', 'lastStatus',
       'bag', 'review', 'ano', 'mes', 'dia', 'diaDaSemana', 'mode',
       'scheduled', 'tippable', 'indoorTipEnabled', 'trackable', 'boxable',
       'placedAtBox', 'reviewed', 'darkKitchen', 'cancellation.code',
       'cancellation.reason', 'expectedDeliveryTime', 'expectedDuration',
       'address.city', 'address.country', 'address.neighborhood',
       'address.state', 'address.streetName', 'address.streetNumber',
       'address.coordinates.latitude', 'address.coordinates.longitude',
       'address.complement', 'driver.id', 'driver.name', 'driver.photoUrl',
       'driver.modal', 'estimatedTimeOfArrival.deliversAt',
       'estimatedTimeOfArrival.updatedAt', 'estimatedTimeOfArrival.reason',
       'id', 'name', 'phoneNumber', 'logo', 'companyGroup', 'type',
       'address.establishment', 'r.adress.city', 'r.address.country',
       'r.address.neighborhood', 'r.address.state', 'r.address.streetName',
  

In [47]:
ifood.head(1)

,id,shortId,createdAt,updatedAt,closedAt,lastStatus,bag,review,ano,mes,dia,diaDaSemana,mode,scheduled,tippable,indoorTipEnabled,trackable,boxable,placedAtBox,reviewed,darkKitchen,cancellation.code,cancellation.reason,expectedDeliveryTime,expectedDuration,address.city,address.country,address.neighborhood,address.state,address.streetName,address.streetNumber,address.coordinates.latitude,address.coordinates.longitude,address.complement,driver.id,driver.name,driver.photoUrl,driver.modal,estimatedTimeOfArrival.deliversAt,estimatedTimeOfArrival.updatedAt,estimatedTimeOfArrival.reason,id,name,phoneNumber,logo,companyGroup,type,address.establishment,r.adress.city,r.address.country,r.address.neighborhood,r.address.state,r.address.streetName,r.address.streetNumber,r.address.coordinates.latitude,r.address.coordinates.longitude,r.address.reference,r.address.complement,methods,total.currency,total.value,platform,appName,appVersion,id,method.name,method.description,type.name,type.description,brand.id,brand.image,brand.name,brand.description,amount.currency,amount.value,credit.cardNumber,debit.cardNumber
0,5f2c646e-2e52-497f-8fa3-5d642f43ce45,2919873206,2021-02-25 12:22:01-03:00,2021-02-25T12:40:32-03:00,2021-02-25T12:39:21-03:00,CONCLUDED,"{'benefits': [], 'deliveryFee': {'value': 549,...",NaN,2021,February,25,Thursday,DELIVERY,False,True,False,True,False,False,False,False,NaN,NaN,2021-02-25T12:43:01-03:00,1260,Ferraz de Vasconcelos,BR,Vila Andeyara,SP,Praça da Biblia,44,-23.540557,-46.369043,,050f3047-2878-4a80-b56b-5a0bb3c349b9,Gildarle Silva,https://nv-production-logistics-driver-account...,MOTORCYCLE,2021-02-25T12:43:01-03:00,2021-02-25T12:40:32-03:00,NaN,84d619e6-9c4c-44a7-83c3-e52632f80e52,Spumoni Restaurante e Sorveteria,40030053,84d619e6-9c4c-44a7-83c3-e52632f80e52/201909261...,IFD,RESTAURANT,,POA,BR,VILA ACOREANA,SP,R RAIMUNDO CORREIA,38,-23.517951,-46.341367,,,"{'id': '83e91c55-f778-430f-a13a-0467508fd9df',...",BRL,2549,ANDROID,iFood,9.90.0,83e91c55-f778-430f-a13a-0467508fd9df,MEAL_VOUCHER,Vale-refeição,ONLINE,Pago pelo app,07b1c8c2-0e38-424e-9fa2-6ea70e5f1dd3,07b1c8c2-0e38-424e-9fa2-6ea70e5f1dd3.png,VR,VR refeição,BRL,2549,NaN,NaN


In [48]:
ifood.drop(labels=['debit.cardNumber','credit.cardNumber','brand.image','brand.id','method.name','appVersion','appName','methods'],axis=1, inplace=True)

In [49]:
ifood.head(1)

,id,shortId,createdAt,updatedAt,closedAt,lastStatus,bag,review,ano,mes,dia,diaDaSemana,mode,scheduled,tippable,indoorTipEnabled,trackable,boxable,placedAtBox,reviewed,darkKitchen,cancellation.code,cancellation.reason,expectedDeliveryTime,expectedDuration,address.city,address.country,address.neighborhood,address.state,address.streetName,address.streetNumber,address.coordinates.latitude,address.coordinates.longitude,address.complement,driver.id,driver.name,driver.photoUrl,driver.modal,estimatedTimeOfArrival.deliversAt,estimatedTimeOfArrival.updatedAt,estimatedTimeOfArrival.reason,id,name,phoneNumber,logo,companyGroup,type,address.establishment,r.adress.city,r.address.country,r.address.neighborhood,r.address.state,r.address.streetName,r.address.streetNumber,r.address.coordinates.latitude,r.address.coordinates.longitude,r.address.reference,r.address.complement,total.currency,total.value,platform,id,method.description,type.name,type.description,brand.name,brand.description,amount.currency,amount.value
0,5f2c646e-2e52-497f-8fa3-5d642f43ce45,2919873206,2021-02-25 12:22:01-03:00,2021-02-25T12:40:32-03:00,2021-02-25T12:39:21-03:00,CONCLUDED,"{'benefits': [], 'deliveryFee': {'value': 549,...",NaN,2021,February,25,Thursday,DELIVERY,False,True,False,True,False,False,False,False,NaN,NaN,2021-02-25T12:43:01-03:00,1260,Ferraz de Vasconcelos,BR,Vila Andeyara,SP,Praça da Biblia,44,-23.540557,-46.369043,,050f3047-2878-4a80-b56b-5a0bb3c349b9,Gildarle Silva,https://nv-production-logistics-driver-account...,MOTORCYCLE,2021-02-25T12:43:01-03:00,2021-02-25T12:40:32-03:00,NaN,84d619e6-9c4c-44a7-83c3-e52632f80e52,Spumoni Restaurante e Sorveteria,40030053,84d619e6-9c4c-44a7-83c3-e52632f80e52/201909261...,IFD,RESTAURANT,,POA,BR,VILA ACOREANA,SP,R RAIMUNDO CORREIA,38,-23.517951,-46.341367,,,BRL,2549,ANDROID,83e91c55-f778-430f-a13a-0467508fd9df,Vale-refeição,ONLINE,Pago pelo app,VR,VR refeição,BRL,2549


In [50]:
ifood['name'].value_counts()

Spumoni Restaurante e Sorveteria                     21
Sodiê Doces - Ferraz de Vasconcelos Vila Açoreana    10
Burger King  - Av. Prof Luiz C Bueno                  8
Habib's - Itaquaquecetuba II                          7
Burguer Channel                                       7
Doce Magia Ferraz de Vasconcelos                      5
Brasileirinho Delivery - Ferraz de Vasconcelos        5
Pizzaria Donatello                                    4
Loop São Paulo Moema 2                                2
Mcdonald's  - Drive Suzano                            2
Three Burger                                          2
Sodiê Doces - São Caetano Centro                      1
McDonald's - Drive Itaquaquecetuba (iba)              1
Mega Esfiha Kemel                                     1
Loop Moema                                            1
Banks Burger                                          1
Cacau Show - Super Higas Itaim Paul                   1
Mcdonald's - Ferraz de Vasconcelos              

In [51]:
ifood['driver.name'].value_counts()

Jefferson Moura      2
Alan Melo            2
Matheus Costa        2
Jefferson Silva      2
Vitor Ferreira       1
José Alves           1
Jardel Costa         1
Jeferson Lora        1
Fabio Soares         1
Alex Santos          1
Samuel Silva         1
Gabriel Fernandes    1
Humberto Portela     1
Guilherme Sous       1
Roberto Conceição    1
Pablo Cardoso        1
Renato Melo          1
Leandro Moraes       1
Gustavo Bravo        1
Sinésio Lima         1
Luciano Alkimim      1
Ricardo Oliveira     1
Luccas Compri        1
Luiz Souza           1
Solange Oliveira     1
Gildarle Silva       1
Amerson Silva        1
Ademario Junior      1
Jefferson Santos     1
Vani Damasceno       1
Flávio Silva         1
Marcos Silva         1
Carlos Zapala        1
Alecio Araújo        1
Humberto Diniz       1
Danilo Leite         1
Walace Venancio      1
Name: driver.name, dtype: int64

In [52]:
delivery['address.city'].value_counts()

Ferraz de Vasconcelos    75
São Paulo                 3
Poa                       3
Sao Paulo                 1
Name: address.city, dtype: int64

In [53]:
ifood['total.value'].sum()/100

6358.78

In [54]:
ifood['diaDaSemana'].value_counts()

Friday       16
Monday       16
Thursday     13
Tuesday      12
Wednesday     9
Sunday        8
Saturday      8
Name: diaDaSemana, dtype: int64

## DF Bag_Itens

In [55]:
bag_itens

,id,uniqueId,externalId,name,description,quantity,subItems,tags,totalPrice,totalPriceWithDiscount,unitPrice,unitPriceWithDiscount,notes
0,f7e42041-6b2c-4b33-94ac-464117949826,ceb42ac6-5983-4d45-89ee-031c9babeb3f,267804,Picadinho,"Arroz, feijão, picadinho e salada.\n+ (1 Bola ...",1,[{'id': '19ada6bc-aa18-4a6e-8f6e-7b3e059e5db9'...,"[TAG::DIADOCLIENTE_ACAI_DOCES_15, TAG::ENTREGA...",2000,2000,1500,1500,NaN
1,a67f0608-2a70-4ec8-bcb8-75984a14bd7c,417b16ca-6d9e-497f-9c68-8976bea3840e,529880001,2 Sanduíches + Batata + Bebida,2 sanduíches (Whopper ou Cheeseburger Duplo B...,3,[{'id': '57632132-e582-4363-b74a-c2f0f8c363ad'...,[],11670,11670,3890,3890,NaN
2,df6ea899-8f57-468a-ba04-9cd4711998da,6b5bd717-c787-429f-8d72-22f745340deb,264996,Parmegiana de frango ou carne,"Arroz, feijão, fritas e salada.\n+(1 Bola de a...",2,[{'id': '277668c4-265e-4530-981b-d5ebc387098e'...,"[TAG::ENTREGA_FOGUETE_NOV19, TAG::MARMITA_FORA...",4000,4000,2000,2000,NaN
3,f7e42041-6b2c-4b33-94ac-464117949826,3502ffc6-e850-4306-921e-97f3ee3b52e3,267804,Picadinho,"Arroz, feijão, picadinho e salada.\n+ (1 Bola ...",1,[{'id': '19ada6bc-aa18-4a6e-8f6e-7b3e059e5db9'...,"[TAG::DIADOCLIENTE_ACAI_DOCES_15, TAG::ENTREGA...",2000,2000,1500,1500,NaN
4,fa51badc-23d6-45a0-91f9-2dc17b04aa19,5f3eba1f-e07f-4dd7-8978-3e6a7319115e,267561,Filé de peixe,"Filé de peixe com batata fritas, arroz, feijão...",1,[{'id': '5dcce42d-5d2a-45b8-a259-5e6018bd8b5f'...,"[TAG::ENTREGA_FOGUETE_NOV19, TAG::MARMITA_FORA...",1800,1800,1800,1800,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,NaN,NaN,70613595,Talharim à bolonhesa 700 gramas,NaN,1,"[{'externalId': '70613624', 'name': 'Não desej...",[TAG::BEST_SELLERS],2699,2699,2699,2699,NaN
143,NaN,NaN,70610541,Parmegiana de frango 700 gramas,NaN,1,"[{'externalId': '70610571', 'name': 'Feijão ca...","[TAG::CRM_MULTI_CAT_BRA, TAG::BEST_SELLERS]",3398,3398,2699,2699,NaN
144,NaN,NaN,9,Linguiça toscana a brasileira,NaN,1,[],"[TAG::ALMOCO_ATE_15, TAG::HUB_2611, TAG::HUB_G...",1099,1099,1099,1099,NaN
145,NaN,NaN,9,Linguiça toscana a brasileira,NaN,1,[],"[TAG::ALMOCO_ATE_15, TAG::HUB_2611, TAG::HUB_G...",990,990,990,990,NaN


dropando as colunas subitens e tags para em seguida dropar as duplicatas


In [56]:
bag_itens.drop(labels=['subItems','tags'],axis=1,inplace=True)

In [57]:
bag_itens.drop_duplicates(keep="last",inplace=True)

In [58]:
bag_itens['name'].value_counts().head(10)

GRANDE                            6
Picadinho                         5
37. Abacaxi com coco              5
06. Brigadeiro 2                  5
2 sanduíches + batata + bebida    4
Virado á Paulista                 4
Escolha 10 itens variados         3
O poderoso chefinho               3
Parmegiana  de frango ou carne    3
2 por R$19,90                     3
Name: name, dtype: int64

In [59]:
bag_itens.shape

(123, 11)

## DF Payments

In [60]:
payments_methods.head(1)

,id,transactions,method.name,method.description,type.name,type.description,brand.id,brand.image,brand.name,brand.description,mealVoucher.cardNumber,amount.currency,amount.value,credit.cardNumber,digitalWallet.id,digitalWallet.name,digitalWallet.description,digitalWallet.image,digitalWallet.cardNumber,digitalWallet.usedBalance.currency,digitalWallet.usedBalance.value,debit.cardNumber
0,83e91c55-f778-430f-a13a-0467508fd9df,"[{'id': '64027405615-640274-004836', 'type': '...",MEAL_VOUCHER,Vale-refeição,ONLINE,Pago pelo app,07b1c8c2-0e38-424e-9fa2-6ea70e5f1dd3,07b1c8c2-0e38-424e-9fa2-6ea70e5f1dd3.png,VR,VR refeição,****************,BRL,2549,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
payments_methods.drop(labels='transactions',axis=1,inplace=True)

In [62]:
payments_methods['method.description'].value_counts()

Vale-refeição       45
Crédito             31
Carteira digital     3
Débito               2
MEAL_VOUCHER         1
Name: method.description, dtype: int64

# <font color = Red>Netflix

## Carregando o DF Histórico de Visualização

In [63]:
data_file = '/content/drive/MyDrive/3_VAMO_AI/01_Case_Ifood/netflix/CONTENT_INTERACTION/ViewingActivity.csv'
data_file_delimiter=','
largest_column_count = 0

In [64]:
with open(data_file, 'r',encoding= 'gb18030') as temp_f:
    lines = temp_f.readlines()
    lines2=[i.split(',')for i in lines]

## Criando o DF pra colocar as linhas com 10 colunas

In [65]:
df = pd.DataFrame(columns=[i for i in range(0, 10)])

In [66]:
for col in range(0,len(lines2)):
  if len(lines2[col])==10:
    df.loc[col]=lines2[col]

In [67]:
df.shape

(13120, 10)

In [68]:
df.head(10)

,0,1,2,3,4,5,6,7,8,9
0,Profile Name,Start Time,Duration,Attributes,Title,Supplemental Video Type,Device Type,Bookmark,Latest Bookmark,Country;;;;;;;;;;;;\n
1,Usuário 1,2021-01-04 17:40:58,00:01:01,,Em Busca do Vale Encantado,,Samsung Novatek DTV Smart TV,00:01:01,00:01:01,BR (Brazil);;;;;;;;;;;;\n
2,Usuário 1,2021-01-04 17:32:40,00:06:04,,Madagascar 2,,Samsung Novatek DTV Smart TV,00:59:49,00:59:49,BR (Brazil);;;;;;;;;;;;\n
3,Usuário 1,2021-01-04 14:19:55,00:18:59,Autoplayed: user action: Unspecified;,A Estrela de Bel茅m,,AOC AOC 2016 MT5800 with no FRC Smart TV,00:18:59,00:18:59,BR (Brazil);;;;;;;;;;;\n
4,Usuário 1,2020-10-23 22:00:42,01:32:46,,A Caminho da Lua,,Samsung Novatek DTV Smart TV,01:30:01,01:30:01,BR (Brazil);;;;;;;;;;;;\n
5,Usuário 1,2020-09-02 14:28:34,00:00:10,,Amaz么nia,,DefaultWidevineAndroidPhone,00:00:03,00:00:03,BR (Brazil);;;;;;;;;;;;\n
6,Usuário 1,2020-08-02 21:34:19,00:47:35,Autoplayed: user action: Unspecified;,Super Monstros - Superamigos para Sempre,,MStar PureTurnkey51 AOC OEM2K Smart TV,00:47:35,00:47:35,BR (Brazil);;;;;;;;;;;\n
7,Usuário 1,2020-08-02 21:33:55,00:00:15,Autoplayed: user action: None;,Super Monstros - Superamigos para Sempre,,MStar PureTurnkey51 AOC OEM2K Smart TV,00:00:15,Not latest view,BR (Brazil);;;;;;;;;;;\n
8,Usuário 1,2020-08-02 21:07:33,00:22:26,,Super Monstros: Turma Nova,,MStar PureTurnkey51 AOC OEM2K Smart TV,00:22:26,00:22:26,BR (Brazil);;;;;;;;;;;;\n
9,Usuário 1,2020-08-02 21:07:04,00:00:06,Autoplayed: user action: None;,Trailer: Super Monstros: Turma Nova,TRAILER,MStar PureTurnkey51 AOC OEM2K Smart TV,00:00:06,00:00:06,BR (Brazil);;;;;;;;;;;\n


## Criando o DF para colocar as linhas com 11 colunas

In [69]:
df2 = pd.DataFrame(columns=[i for i in range(0, 11)])

In [70]:
for col in range(0,len(lines2)):
  if len(lines2[col])==11:
    df2.loc[col]=lines2[col]

In [71]:
df2.head()

,0,1,2,3,4,5,6,7,8,9,10
21,Usuário 1,2020-07-27 16:18:30,00:21:46,"Autoplayed: user action: None;""","""""Drag玫es: Corrida at茅 o Limite: Temporada 3: ...","n茫o! (Epis贸dio 12)""""",,MStar PureTurnkey51 AOC OEM2K Smart TV,00:22:27,00:22:27,"BR (Brazil)"";;;;;;;;;;;\n"
48,Usuário 1,2020-07-20 16:08:56,00:07:48,"Autoplayed: user action: None;""","""""Shaun",o Carneiro: Temporada 3: O jogo / Vamos ver u...,,MStar PureTurnkey51 AOC OEM2K Smart TV,00:08:28,00:08:28,"BR (Brazil)"";;;;;;;;;;;\n"
51,"""Usuário 1",2020-07-20 15:31:58,00:12:16,,"""""Shaun",o Carneiro: Temporada 3: Os bagunceiros / O c...,,MStar PureTurnkey51 AOC OEM2K Smart TV,00:12:56,00:12:56,"BR (Brazil)"";;;;;;;;;;;;\n"
52,Usuário 1,2020-07-20 15:28:30,00:03:22,"Autoplayed: user action: Unspecified;""","""""Shaun",o Carneiro: Temporada 3: O pastor / Voc锚 esqu...,,MStar PureTurnkey51 AOC OEM2K Smart TV,00:12:47,00:12:47,"BR (Brazil)"";;;;;;;;;;;\n"
53,"""Usuário 1",2020-07-20 00:36:25,00:08:45,,"""""Shaun",o Carneiro: Temporada 3: O pastor / Voc锚 esqu...,,MStar PureTurnkey51 AOC OEM2K Smart TV,00:09:25,Not latest view,"BR (Brazil)"";;;;;;;;;;;;\n"


## Juntando a coluna 4 com a coluna 5, que causava o erro em algumas linhas

In [72]:
df2[4] = df2[4].str.cat(df2[5])

Tirando a coluna 5

In [73]:
df2.drop(columns=5,inplace=True)

## Renomeando as colunas que restaram para ficar igual ao DF com 10 colunas

In [74]:
df2.rename(columns={6:5,7:6,8:7,9:8,10:9},inplace=True)

## Concatenando os dataframes

In [75]:
df=pd.concat([df,df2])

In [76]:
df.shape

(13559, 10)

## Alguns caracteres vieram desconfigurados substitui pelos corretos

In [77]:
df = df.replace('贸','ó', regex=True)
df = df.replace('茅','é', regex=True)
df = df.replace('么','ô',regex=True)
df=df.replace('脷','ú',regex=True)
df=df.replace('莽','ç' ,regex=True)
df=df.replace('茫', 'ã', regex=True)
df=df.replace('铆','í',regex=True)
df=df.replace('脿','à',regex=True)
df=df.replace('谩','á', regex=True)

## Criando o df final com o header correto

In [78]:
headers= df.iloc[0]
historico_visu=pd.DataFrame(df.values[1:],columns=headers)

In [79]:
historico_visu.head()

,Profile Name,Start Time,Duration,Attributes,Title,Supplemental Video Type,Device Type,Bookmark,Latest Bookmark,Country;;;;;;;;;;;;\n
0,Usuário 1,2021-01-04 17:40:58,00:01:01,,Em Busca do Vale Encantado,,Samsung Novatek DTV Smart TV,00:01:01,00:01:01,BR (Brazil);;;;;;;;;;;;\n
1,Usuário 1,2021-01-04 17:32:40,00:06:04,,Madagascar 2,,Samsung Novatek DTV Smart TV,00:59:49,00:59:49,BR (Brazil);;;;;;;;;;;;\n
2,Usuário 1,2021-01-04 14:19:55,00:18:59,Autoplayed: user action: Unspecified;,A Estrela de Belém,,AOC AOC 2016 MT5800 with no FRC Smart TV,00:18:59,00:18:59,BR (Brazil);;;;;;;;;;;\n
3,Usuário 1,2020-10-23 22:00:42,01:32:46,,A Caminho da Lua,,Samsung Novatek DTV Smart TV,01:30:01,01:30:01,BR (Brazil);;;;;;;;;;;;\n
4,Usuário 1,2020-09-02 14:28:34,00:00:10,,Amazônia,,DefaultWidevineAndroidPhone,00:00:03,00:00:03,BR (Brazil);;;;;;;;;;;;\n


In [80]:
historico_visu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13558 entries, 0 to 13557
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Profile Name             13558 non-null  object
 1   Start Time               13558 non-null  object
 2   Duration                 13558 non-null  object
 3   Attributes               13558 non-null  object
 4   Title                    13558 non-null  object
 5   Supplemental Video Type  13558 non-null  object
 6   Device Type              13558 non-null  object
 7   Bookmark                 13558 non-null  object
 8   Latest Bookmark          13558 non-null  object
 9   Country;;;;;;;;;;;;
     13558 non-null  object
dtypes: object(10)
memory usage: 1.0+ MB


## Corrigindo uns caracteres que vieram a mais

In [81]:
historico_visu.rename(columns={'Country;;;;;;;;;;;;\n':'Country'},inplace=True)

In [82]:
historico_visu.shape

(13558, 10)

In [83]:
historico_visu['Country']=historico_visu['Country'].astype('string')

In [84]:
historico_visu['Country']=historico_visu['Country'].apply(lambda x: x.replace(';',''))

In [85]:
historico_visu['Country']=historico_visu['Country'].apply(lambda x: x.replace('"',''))

In [86]:
historico_visu['Profile Name']=historico_visu['Profile Name'].apply(lambda x: x.replace('"',''))

In [87]:
historico_visu['Country'].value_counts()

BR (Brazil)\n           13458
IE (Ireland)\n             98
FR (France)\n               1
US (United States)\n        1
Name: Country, dtype: int64

In [88]:
historico_visu['Profile Name'].value_counts()

Usuário 2    9250
Usuário 3    3759
Usuário 1     549
Name: Profile Name, dtype: int64

## Vendo os perfis que existem na conta


In [89]:
historico_visu['Profile Name'].value_counts()

Usuário 2    9250
Usuário 3    3759
Usuário 1     549
Name: Profile Name, dtype: int64

## Qual filme as contas mais viram?


In [90]:
historico_visu['Title'].value_counts()

Até o último Homem                                           23
Homem de Ferro 2                                             22
Velozes e Furiosos 3 鈥?Desafio em Tóquio                     22
Velozes & Furiosos 6                                         21
Os Vingadores - The Avengers                                 20
                                                             ..
Soul Surfer - Coragem de Viver                                1
The Flash: Temporada 3: Rua Infantino (Episódio 22)           1
O Gambito da Rainha: Minissérie: Treinamento (Episódio 5)     1
AIB Hook 1                                                    1
Lucifer: Temporada 2: Partes íntimas (Episódio 4)             1
Name: Title, Length: 6313, dtype: int64

## Qual pais mais assistiram



In [91]:
historico_visu.head(1)

,Profile Name,Start Time,Duration,Attributes,Title,Supplemental Video Type,Device Type,Bookmark,Latest Bookmark,Country
0,Usuário 1,2021-01-04 17:40:58,00:01:01,,Em Busca do Vale Encantado,,Samsung Novatek DTV Smart TV,00:01:01,00:01:01,BR (Brazil)\n


In [92]:
historico_visu['Country'].value_counts()

BR (Brazil)\n           13458
IE (Ireland)\n             98
FR (France)\n               1
US (United States)\n        1
Name: Country, dtype: int64

In [93]:
historico_visu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13558 entries, 0 to 13557
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Profile Name             13558 non-null  object
 1   Start Time               13558 non-null  object
 2   Duration                 13558 non-null  object
 3   Attributes               13558 non-null  object
 4   Title                    13558 non-null  object
 5   Supplemental Video Type  13558 non-null  object
 6   Device Type              13558 non-null  object
 7   Bookmark                 13558 non-null  object
 8   Latest Bookmark          13558 non-null  object
 9   Country                  13558 non-null  object
dtypes: object(10)
memory usage: 1.0+ MB


## Limpando os titulos

In [94]:
historico_visu['clean_title'] = historico_visu['Title'].apply(clean_title)

## Convertendo duração em timedelta para fazer a soma

In [95]:
historico_visu['Duration']= pd.to_timedelta(historico_visu['Duration'])

In [96]:
historico_visu['Duration'].sum()

Timedelta('197 days 19:40:55')

In [97]:
historico_visu['segundos']=historico_visu['Duration'].dt.total_seconds()

In [98]:
historico_visu.head()

,Profile Name,Start Time,Duration,Attributes,Title,Supplemental Video Type,Device Type,Bookmark,Latest Bookmark,Country,clean_title,segundos
0,Usuário 1,2021-01-04 17:40:58,0 days 00:01:01,,Em Busca do Vale Encantado,,Samsung Novatek DTV Smart TV,00:01:01,00:01:01,BR (Brazil)\n,Em Busca do Vale Encantado,61.0
1,Usuário 1,2021-01-04 17:32:40,0 days 00:06:04,,Madagascar 2,,Samsung Novatek DTV Smart TV,00:59:49,00:59:49,BR (Brazil)\n,Madagascar 2,364.0
2,Usuário 1,2021-01-04 14:19:55,0 days 00:18:59,Autoplayed: user action: Unspecified;,A Estrela de Belém,,AOC AOC 2016 MT5800 with no FRC Smart TV,00:18:59,00:18:59,BR (Brazil)\n,A Estrela de Belém,1139.0
3,Usuário 1,2020-10-23 22:00:42,0 days 01:32:46,,A Caminho da Lua,,Samsung Novatek DTV Smart TV,01:30:01,01:30:01,BR (Brazil)\n,A Caminho da Lua,5566.0
4,Usuário 1,2020-09-02 14:28:34,0 days 00:00:10,,Amazônia,,DefaultWidevineAndroidPhone,00:00:03,00:00:03,BR (Brazil)\n,Amazônia,10.0


## Mudando o Start Time pra datetime, para separar o ano, mes,dia e dia da semana.

In [99]:
historico_visu['Start Time'] = pd.to_datetime(historico_visu['Start Time'], utc=True)

In [100]:
historico_visu['year'] = pd.DatetimeIndex(historico_visu['Start Time']).year
historico_visu['month'] = pd.DatetimeIndex(historico_visu['Start Time']).month
historico_visu['day'] = pd.DatetimeIndex(historico_visu['Start Time']).day
historico_visu['day.name'] = historico_visu['Start Time'].dt.day_name()
historico_visu.head()

,Profile Name,Start Time,Duration,Attributes,Title,Supplemental Video Type,Device Type,Bookmark,Latest Bookmark,Country,clean_title,segundos,year,month,day,day.name
0,Usuário 1,2021-01-04 17:40:58+00:00,0 days 00:01:01,,Em Busca do Vale Encantado,,Samsung Novatek DTV Smart TV,00:01:01,00:01:01,BR (Brazil)\n,Em Busca do Vale Encantado,61.0,2021,1,4,Monday
1,Usuário 1,2021-01-04 17:32:40+00:00,0 days 00:06:04,,Madagascar 2,,Samsung Novatek DTV Smart TV,00:59:49,00:59:49,BR (Brazil)\n,Madagascar 2,364.0,2021,1,4,Monday
2,Usuário 1,2021-01-04 14:19:55+00:00,0 days 00:18:59,Autoplayed: user action: Unspecified;,A Estrela de Belém,,AOC AOC 2016 MT5800 with no FRC Smart TV,00:18:59,00:18:59,BR (Brazil)\n,A Estrela de Belém,1139.0,2021,1,4,Monday
3,Usuário 1,2020-10-23 22:00:42+00:00,0 days 01:32:46,,A Caminho da Lua,,Samsung Novatek DTV Smart TV,01:30:01,01:30:01,BR (Brazil)\n,A Caminho da Lua,5566.0,2020,10,23,Friday
4,Usuário 1,2020-09-02 14:28:34+00:00,0 days 00:00:10,,Amazônia,,DefaultWidevineAndroidPhone,00:00:03,00:00:03,BR (Brazil)\n,Amazônia,10.0,2020,9,2,Wednesday


In [101]:
historico_visu["month"].replace(month, inplace=True)
historico_visu.head()

,Profile Name,Start Time,Duration,Attributes,Title,Supplemental Video Type,Device Type,Bookmark,Latest Bookmark,Country,clean_title,segundos,year,month,day,day.name
0,Usuário 1,2021-01-04 17:40:58+00:00,0 days 00:01:01,,Em Busca do Vale Encantado,,Samsung Novatek DTV Smart TV,00:01:01,00:01:01,BR (Brazil)\n,Em Busca do Vale Encantado,61.0,2021,January,4,Monday
1,Usuário 1,2021-01-04 17:32:40+00:00,0 days 00:06:04,,Madagascar 2,,Samsung Novatek DTV Smart TV,00:59:49,00:59:49,BR (Brazil)\n,Madagascar 2,364.0,2021,January,4,Monday
2,Usuário 1,2021-01-04 14:19:55+00:00,0 days 00:18:59,Autoplayed: user action: Unspecified;,A Estrela de Belém,,AOC AOC 2016 MT5800 with no FRC Smart TV,00:18:59,00:18:59,BR (Brazil)\n,A Estrela de Belém,1139.0,2021,January,4,Monday
3,Usuário 1,2020-10-23 22:00:42+00:00,0 days 01:32:46,,A Caminho da Lua,,Samsung Novatek DTV Smart TV,01:30:01,01:30:01,BR (Brazil)\n,A Caminho da Lua,5566.0,2020,October,23,Friday
4,Usuário 1,2020-09-02 14:28:34+00:00,0 days 00:00:10,,Amazônia,,DefaultWidevineAndroidPhone,00:00:03,00:00:03,BR (Brazil)\n,Amazônia,10.0,2020,September,2,Wednesday


In [102]:
historico_visu['part_of_day'] = historico_visu['Start Time'].dt.hour.apply(get_part_of_day)

In [103]:
historico_visu['month'].value_counts()

December     1550
January      1443
July         1259
August       1243
February     1238
November     1057
April        1009
September     978
March         968
June          963
October       960
May           890
Name: month, dtype: int64

## Vendo qual o dia da semana os perfis mais assistem

In [104]:
historico_visu['day.name'].value_counts()

Sunday       3174
Saturday     2751
Friday       1796
Tuesday      1541
Monday       1511
Thursday     1428
Wednesday    1357
Name: day.name, dtype: int64

## Analisando o Usuário 3

In [105]:
historico_3=historico_visu.loc[(historico_visu['Profile Name']=='Usuário 3')]
historico_3.head()

,Profile Name,Start Time,Duration,Attributes,Title,Supplemental Video Type,Device Type,Bookmark,Latest Bookmark,Country,clean_title,segundos,year,month,day,day.name,part_of_day
515,Usuário 3,2021-02-23 03:42:48+00:00,0 days 00:00:01,,The Crown: Temporada 1: Gloriana (Episódio 10),,DefaultWidevineAndroidPhone,00:00:01,00:00:01,BR (Brazil)\n,The Crown,1.0,2021,February,23,Tuesday,Late Night
516,Usuário 3,2021-02-21 17:19:43+00:00,0 days 00:00:08,Autoplayed: user action: None;,Waterworld_hook_01_16x9,HOOK,Samsung 2016 Jazz-L UHD TV Smart TV,00:00:08,00:00:08,BR (Brazil)\n,Waterworld_hook_01_16x9,8.0,2021,February,21,Sunday,Evening
517,Usuário 3,2021-02-20 19:55:59+00:00,0 days 00:46:47,Autoplayed: user action: Unspecified;,O Irland锚s,,Samsung 2016 Jazz-L UHD TV Smart TV,03:21:08,03:21:08,BR (Brazil)\n,O Irland锚s,2807.0,2021,February,20,Saturday,Evening
518,Usuário 3,2021-02-20 18:07:31+00:00,0 days 01:47:40,Autoplayed: user action: Unspecified;,A Escavação,,Samsung 2016 Jazz-L UHD TV Smart TV,01:47:40,01:47:40,BR (Brazil)\n,A Escavação,6460.0,2021,February,20,Saturday,Evening
519,Usuário 3,2021-02-20 18:06:49+00:00,0 days 00:00:17,Autoplayed: user action: Unspecified;,Ponto Vermelho,,Samsung 2016 Jazz-L UHD TV Smart TV,00:00:17,00:00:17,BR (Brazil)\n,Ponto Vermelho,17.0,2021,February,20,Saturday,Evening


In [106]:
historico_3['Title'].value_counts()

Lucifer: Temporada 1: Piloto (Episódio 1)                                                  13
Black Mirror: Temporada 3: Queda livre (Episódio 1)                                        12
Temporada 1 (Trailer): Expresso do Amanhã                                                  12
Stranger Things: Temporada 1: Capítulo um: O desaparecimento de Will Byers (Episódio 1)    10
Outra Vida: Temporada 1 (Trailer)                                                          10
                                                                                           ..
The Great Wall_hook_primary_16x9                                                            1
Lost: Temporada 1: O vigarista (Episódio 8)                                                 1
Stranger Things: Stranger Things 2: Capítulo tr锚s: O girino (Episódio 3)                    1
A Anatomia de Grey: Temporada 5: Luz e sombras (Episódio 17)                                1
Border Security: America's Front Line: Temporada 1: Episódio

In [107]:
historico_3.groupby(['Title']).agg({'Duration':'sum'}).sort_values(by='Duration', ascending=False).head(11)

,Duration
Title,
2012,0 days 06:53:25
Tropa de Elite,0 days 05:27:27
Forrest Gump,0 days 05:01:21
Invencível,0 days 04:25:19
Prenda-Me Se for Capaz,0 days 04:21:58
A Grande Aposta,0 days 04:07:03
Terra à Deriva,0 days 04:00:03
Impacto Profundo,0 days 04:00:00
Oblivion,0 days 03:59:29


In [108]:
historico_3.shape

(3759, 17)

# <font color=red> Netflix Natascha

## Carregando o df

In [109]:
historico_visu_nat=pd.read_csv('/content/drive/MyDrive/3_VAMO_AI/Dados_Natascha/NEtflix/CONTENT_INTERACTION/ViewingActivity.csv')

In [110]:
historico_visu_nat.head()

,Profile Name,Start Time,Duration,Attributes,Title,Supplemental Video Type,Device Type,Bookmark,Latest Bookmark,Country
0,Alesandra,2021-07-19 03:20:14,00:37:00,Autoplayed: user action: None;,"Roubos Inacreditáveis: Temporada 1: Sexo, dinh...",NaN,Samsung 2015 NT14M DTV Smart TV,00:37:04,00:37:04,BR (Brazil)
1,Alesandra,2021-07-19 02:40:47,00:39:12,Autoplayed: user action: User_Interaction;,"Roubos Inacreditáveis: Temporada 1: Sexo, dinh...",NaN,Samsung 2015 NT14M DTV Smart TV,00:39:12,00:39:12,BR (Brazil)
2,Alesandra,2021-07-19 00:48:49,01:49:01,Autoplayed: user action: User_Interaction;,Estômago,NaN,Samsung 2015 NT14M DTV Smart TV,01:49:01,01:49:01,BR (Brazil)
3,Alesandra,2021-07-18 23:16:11,01:25:38,Autoplayed: user action: User_Interaction;,Invasão,NaN,Samsung 2015 NT14M DTV Smart TV,01:25:38,01:25:38,BR (Brazil)
4,Alesandra,2021-07-18 21:17:47,01:44:31,Autoplayed: user action: User_Interaction;,Águas que Corroem,NaN,Samsung 2015 NT14M DTV Smart TV,01:44:06,01:44:06,BR (Brazil)


## Analisando o df

Quais perfis disponíveis

In [111]:
historico_visu_nat['Profile Name'].value_counts()

Cecilia      4366
Alesandra    2829
natascha     2722
kids            3
Vanessa         1
Name: Profile Name, dtype: int64

Títulos mais assistidos pelos perfis

In [112]:
historico_visu_nat['Title'].value_counts()

Wynonna Earp: Temporada 1: Manter o fogo aceso (Episódio 2)                  20
Wynonna Earp: Temporada 1: Enterre-me com minhas armas (Episódio 9)          16
A Anatomia de Grey: Temporada 3: Cicatrizes e lembrancinhas (Episódio 18)    16
Doce vingança                                                                16
Zumbo's Just Desserts: Temporada 2: É hora de fazer mágica! (Episódio 1)     15
                                                                             ..
Two and a Half Men: Temporada 3: A pistola hermafrodita (Episódio 24)         1
Juventude à Flor da Pele: Vol. 4: Freddie (Episódio 5)                        1
Disque Amiga para Matar: Temporada 1: Piloto (Episódio 1)                     1
Fargo: Temporada 2: Esperando pelo holandês (Episódio 1)                      1
O Gambito da Rainha: Minissérie: Aberturas (Episódio 1)                       1
Name: Title, Length: 4430, dtype: int64

Paises dos quais foram reproduzidos os titulos

In [113]:
historico_visu_nat['Country'].value_counts()

BR (Brazil)            9221
IE (Ireland)            677
PT (Portugal)             8
US (United States)        7
GB (United Kingdom)       5
IT (Italy)                1
PL (Poland)               1
ZZ (Unknown Region)       1
Name: Country, dtype: int64

In [114]:
historico_visu_nat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9921 entries, 0 to 9920
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Profile Name             9921 non-null   object
 1   Start Time               9921 non-null   object
 2   Duration                 9921 non-null   object
 3   Attributes               1877 non-null   object
 4   Title                    9921 non-null   object
 5   Supplemental Video Type  395 non-null    object
 6   Device Type              9921 non-null   object
 7   Bookmark                 9921 non-null   object
 8   Latest Bookmark          9921 non-null   object
 9   Country                  9921 non-null   object
dtypes: object(10)
memory usage: 775.2+ KB


## Convertendo a duração pra timedelta

In [115]:
historico_visu_nat['Duration']= pd.to_timedelta(historico_visu_nat['Duration'])

In [116]:
historico_visu_nat['Duration'].sum()

Timedelta('173 days 14:53:36')

In [117]:
historico_visu_nat['segundos']=historico_visu_nat['Duration'].dt.total_seconds()

In [118]:
historico_visu_nat.head()

,Profile Name,Start Time,Duration,Attributes,Title,Supplemental Video Type,Device Type,Bookmark,Latest Bookmark,Country,segundos
0,Alesandra,2021-07-19 03:20:14,0 days 00:37:00,Autoplayed: user action: None;,"Roubos Inacreditáveis: Temporada 1: Sexo, dinh...",NaN,Samsung 2015 NT14M DTV Smart TV,00:37:04,00:37:04,BR (Brazil),2220.0
1,Alesandra,2021-07-19 02:40:47,0 days 00:39:12,Autoplayed: user action: User_Interaction;,"Roubos Inacreditáveis: Temporada 1: Sexo, dinh...",NaN,Samsung 2015 NT14M DTV Smart TV,00:39:12,00:39:12,BR (Brazil),2352.0
2,Alesandra,2021-07-19 00:48:49,0 days 01:49:01,Autoplayed: user action: User_Interaction;,Estômago,NaN,Samsung 2015 NT14M DTV Smart TV,01:49:01,01:49:01,BR (Brazil),6541.0
3,Alesandra,2021-07-18 23:16:11,0 days 01:25:38,Autoplayed: user action: User_Interaction;,Invasão,NaN,Samsung 2015 NT14M DTV Smart TV,01:25:38,01:25:38,BR (Brazil),5138.0
4,Alesandra,2021-07-18 21:17:47,0 days 01:44:31,Autoplayed: user action: User_Interaction;,Águas que Corroem,NaN,Samsung 2015 NT14M DTV Smart TV,01:44:06,01:44:06,BR (Brazil),6271.0


## Convertendo Start Time para date time e em seguida pegar mês, ano, dia e dia da semana

In [119]:
historico_visu_nat['Start Time'] = pd.to_datetime(historico_visu_nat['Start Time'], utc=True)

In [120]:
historico_visu_nat['year'] = pd.DatetimeIndex(historico_visu_nat['Start Time']).year
historico_visu_nat['month'] = pd.DatetimeIndex(historico_visu_nat['Start Time']).month
historico_visu_nat['day'] = pd.DatetimeIndex(historico_visu_nat['Start Time']).day
historico_visu_nat['day.name'] = historico_visu_nat['Start Time'].dt.day_name()
historico_visu_nat.head(1)

,Profile Name,Start Time,Duration,Attributes,Title,Supplemental Video Type,Device Type,Bookmark,Latest Bookmark,Country,segundos,year,month,day,day.name
0,Alesandra,2021-07-19 03:20:14+00:00,0 days 00:37:00,Autoplayed: user action: None;,"Roubos Inacreditáveis: Temporada 1: Sexo, dinh...",NaN,Samsung 2015 NT14M DTV Smart TV,00:37:04,00:37:04,BR (Brazil),2220.0,2021,7,19,Monday


In [121]:
historico_visu_nat["month"].replace(month, inplace=True)
historico_visu_nat.head(1)

,Profile Name,Start Time,Duration,Attributes,Title,Supplemental Video Type,Device Type,Bookmark,Latest Bookmark,Country,segundos,year,month,day,day.name
0,Alesandra,2021-07-19 03:20:14+00:00,0 days 00:37:00,Autoplayed: user action: None;,"Roubos Inacreditáveis: Temporada 1: Sexo, dinh...",NaN,Samsung 2015 NT14M DTV Smart TV,00:37:04,00:37:04,BR (Brazil),2220.0,2021,July,19,Monday


In [122]:
historico_visu_nat['part_of_day'] = historico_visu_nat['Start Time'].dt.hour.apply(get_part_of_day)

In [123]:
historico_visu_nat['month'].value_counts()

April        1212
January      1028
May          1005
March         987
November      863
August        776
December      770
October       744
September     687
February      663
July          607
June          579
Name: month, dtype: int64

In [124]:
historico_visu_nat['day.name'].value_counts()

Sunday       1858
Monday       1662
Saturday     1380
Wednesday    1333
Thursday     1297
Tuesday      1248
Friday       1143
Name: day.name, dtype: int64

## Limpando os Títulos

In [125]:
historico_visu_nat['clean_title'] = historico_visu_nat['Title'].apply(clean_title)

## Separando o meu perfil

In [126]:
historico_nat=historico_visu_nat.loc[(historico_visu_nat['Profile Name']=='natascha')]
historico_nat.head(1)

,Profile Name,Start Time,Duration,Attributes,Title,Supplemental Video Type,Device Type,Bookmark,Latest Bookmark,Country,segundos,year,month,day,day.name,part_of_day,clean_title
7199,natascha,2021-07-09 22:49:40+00:00,0 days 01:39:00,NaN,Rua do Medo: 1994 – Parte 1,NaN,Safari MAC (Cadmium),01:39:00,01:39:00,BR (Brazil),5940.0,2021,July,9,Friday,Night,Rua do Medo


In [127]:
historico_nat['Title'].value_counts()

Wynonna Earp: Temporada 1: Manter o fogo aceso (Episódio 2)                 19
Wynonna Earp: Temporada 1: Enterre-me com minhas armas (Episódio 9)         16
Wynonna Earp: Temporada 2: Vamos fingir que somos estranhos (Episódio 5)    13
Wynonna Earp: Temporada 2: Todo mundo sabe (Episódio 7)                     13
Wynonna Earp: Temporada 1: Jack duas caras (Episódio 8)                     12
                                                                            ..
Os Cavaleiros do Zodíaco: Temporada 2: A ilha do espectro (Episódio 4)       1
Olmo & the Seagull_hook_01_16x9                                              1
Vis a vis: Temporada 2: Prendendo os dedos (Episódio 9)                      1
Hitler's Circle of Evil: Temporada 1: Colapso (Episódio 10)                  1
Investigação Criminal: Temporada 1: Crime da Oscar Freire (Episódio 6)       1
Name: Title, Length: 1450, dtype: int64

In [128]:
historico_nat.groupby(['Title']).agg({'Duration':'sum'}).sort_values(by='Duration', ascending=False).head(11)

,Duration
Title,
Carol,0 days 04:40:39
Harry Potter e as Relíquias da Morte,0 days 04:28:03
Sense8: Temporada 2: Amor vincit omnia (Episódio 12),0 days 04:23:47
Wynonna Earp: Temporada 1: Manter o fogo aceso (Episódio 2),0 days 04:21:54
Comer Rezar Amar,0 days 04:09:03
Um Lugar Chamado Notting Hill,0 days 04:05:20
Brilho Eterno de uma Mente sem Lembranças,0 days 04:03:15
18 Presentes,0 days 03:42:34
O Sorriso de Monalisa,0 days 03:41:08


# <font color= Red>Spotify e API Thiago

## abrindo o arquivo de historico de reprodução


In [129]:
sp_historico=pd.read_json('/content/drive/MyDrive/3_VAMO_AI/01_Case_Ifood/Spotify/StreamingHistory0.json')
sp_historico.head()

,endTime,artistName,trackName,msPlayed
0,2020-01-31 03:00,Clean Bandit,Symphony (feat. Zara Larsson),206101
1,2020-08-26 11:36,Queen,Bohemian Rhapsody - Remastered 2011,354320
2,2020-10-04 22:24,Beau Diako,Kyiv,157585
3,2020-10-04 22:27,invention_,Sonar,148937
4,2020-10-05 00:12,xander.,Where We Met,66004


In [130]:
sp_historico.shape

(22, 4)

## Convertendo para Date Time a coluna end time, para poder quebrar em dias, meses, semanas etc

In [131]:
sp_historico['endTime'] = pd.to_datetime(sp_historico['endTime'])

Aplicando a função get_part_of_day para separar os horarios de reprodução em faixas de horários como night, late night, early morning etc

In [132]:
sp_historico['part_of_day'] = sp_historico['endTime'].dt.hour.apply(get_part_of_day)

Qual o periodo do dia há mais reproduções?

In [133]:
sp_historico['part_of_day'].value_counts()

Night         10
Late Night     9
Evening        2
Morning        1
Name: part_of_day, dtype: int64

Quebrando em ano, mês, dia e dia da semana

In [134]:
sp_historico['year'] = pd.DatetimeIndex(sp_historico['endTime']).year
sp_historico['month'] = pd.DatetimeIndex(sp_historico['endTime']).month
sp_historico['day'] = pd.DatetimeIndex(sp_historico['endTime']).day
sp_historico['day_name'] = sp_historico['endTime'].dt.day_name()
sp_historico.head()

,endTime,artistName,trackName,msPlayed,part_of_day,year,month,day,day_name
0,2020-01-31 03:00:00,Clean Bandit,Symphony (feat. Zara Larsson),206101,Late Night,2020,1,31,Friday
1,2020-08-26 11:36:00,Queen,Bohemian Rhapsody - Remastered 2011,354320,Morning,2020,8,26,Wednesday
2,2020-10-04 22:24:00,Beau Diako,Kyiv,157585,Night,2020,10,4,Sunday
3,2020-10-04 22:27:00,invention_,Sonar,148937,Night,2020,10,4,Sunday
4,2020-10-05 00:12:00,xander.,Where We Met,66004,Late Night,2020,10,5,Monday


Mudando de número do mês para nome do mês

In [135]:
sp_historico["month"].replace(month, inplace=True)


Qual mês há mais reproduções?

In [136]:
sp_historico['month'].value_counts()

November    11
December     6
October      3
August       1
January      1
Name: month, dtype: int64

Qual dia da semana há mais reproduções

In [137]:
sp_historico['day_name'].value_counts()

Wednesday    9
Friday       7
Sunday       4
Monday       1
Thursday     1
Name: day_name, dtype: int64

## Para abrir o arquivo YourLibrary o read_json não funciona, então usei o open e o load pra abrir e transformar o arquivo em dicionário

In [138]:
with open ('/content/drive/MyDrive/3_VAMO_AI/01_Case_Ifood/Spotify/YourLibrary.json',encoding='utf-8') as f:
  biblioteca=json.load(f)
type(biblioteca)

dict

In [139]:
biblioteca.keys()

dict_keys(['tracks', 'albums', 'shows', 'episodes', 'bannedTracks', 'other'])

## Depois de transformar em dicionário, transformo cada índice em um dataframe

In [140]:
faixas=pd.json_normalize(biblioteca['tracks'])
albuns=pd.json_normalize(biblioteca['albums'])
podcasts=pd.json_normalize(biblioteca['shows'])
episodios=pd.json_normalize(biblioteca['episodes'])
faixas_ban=pd.json_normalize(biblioteca['bannedTracks'])
outros=pd.json_normalize(biblioteca['other'])

In [141]:
type(faixas)

pandas.core.frame.DataFrame

In [142]:
lista=[faixas, albuns, podcasts, episodios,outros]
biblioteca=pd.concat(lista,axis=1)

In [143]:
biblioteca.head()

,artist,album,track
0,JAY-Z,Collision Course,Izzo / In the End
1,Sebastian Ingrosso,Calling (Lose My Mind),Calling (Lose My Mind) - Radio Edit
2,Capital Inicial,Rosas e Vinho Tinto,À Sua Maneira (De Música Ligeira)
3,Seu Jorge,"Musicas Para Churrasco, Vol. 1",Amiga Da Minha Mulher
4,Armin van Buuren,Not Giving Up On Love,Not Giving Up On Love - Radio Edit


In [144]:
biblioteca['artist'].value_counts()

Metallica           11
Maroon 5             6
Armin van Buuren     6
Lasgo                5
Calvin Harris        5
                    ..
Bad Meets Evil       1
Jonas Blue           1
K-Pop All-Stars      1
Charli XCX           1
Martin Garrix        1
Name: artist, Length: 196, dtype: int64

## Spotipy

In [145]:
pip install spotipy

In [146]:
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
import sys
import csv
import requests

## Configurando o spotipy

In [147]:
client_id = "24e3a45dbe034db6bec311f26d04c5dd"
client_secret = "4f1dff8480d94c70827082be7a832cb1"

client_credentials_manager = SpotifyClientCredentials(client_id,client_secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [148]:
auth_url = 'https://accounts.spotify.com/api/token'

In [149]:
data = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
}

In [150]:
auth_response = requests.post(auth_url, data=data)

In [151]:
access_token = auth_response.json().get('access_token')

In [152]:
headers = {
    'Authorization': 'Bearer {}'.format(access_token)
}

## Funções para usar o spotipy

In [153]:
def get_artist_id(artist):
    if len(artist) > 1:
      name =artist
    else:
      name = None

    results = sp.search(q='artist:' + name, type='artist')
    items = results['artists']['items']
    if len(items) > 0:
        artist = items[0]
        return artist['id']

In [154]:
def artist_genre(artist):

  search = sp.search(q='artist:'+ artist, type='artist',limit=50)
  
  if search['artists']['items']:
    genres = search['artists']['items'][0]['genres']
    return genres
  else:
    return None 

## Pegando os ids de cada artista

Duplicando o nome do artista pra aplicar a função

In [155]:
biblioteca['artist_id']=biblioteca.loc[:,'artist']


Aplicando a função de id

In [156]:
biblioteca['artist_id']=biblioteca['artist_id'].apply(get_artist_id)

Conferindo se deu resultado

In [157]:
biblioteca.head()

,artist,album,track,artist_id
0,JAY-Z,Collision Course,Izzo / In the End,3nFkdlSjzX9mRTtwJOzDYB
1,Sebastian Ingrosso,Calling (Lose My Mind),Calling (Lose My Mind) - Radio Edit,6hyMWrxGBsOx6sWcVj1DqP
2,Capital Inicial,Rosas e Vinho Tinto,À Sua Maneira (De Música Ligeira),4Z0yuwHVJBROVZqFpTIr0d
3,Seu Jorge,"Musicas Para Churrasco, Vol. 1",Amiga Da Minha Mulher,0i1s9WcIu0PrUvHzALgofo
4,Armin van Buuren,Not Giving Up On Love,Not Giving Up On Love - Radio Edit,0SfsnGyD8FpIN4U4WCkBZ5


## Pegando o Gênero de cada artista

In [158]:
biblioteca['genre']=biblioteca.loc[:,'artist']

In [159]:
biblioteca['genre']=biblioteca['genre'].apply(artist_genre)

In [160]:
biblioteca.head()

,artist,album,track,artist_id,genre
0,JAY-Z,Collision Course,Izzo / In the End,3nFkdlSjzX9mRTtwJOzDYB,"[east coast hip hop, hip hop, pop rap, rap]"
1,Sebastian Ingrosso,Calling (Lose My Mind),Calling (Lose My Mind) - Radio Edit,6hyMWrxGBsOx6sWcVj1DqP,"[dance pop, edm, electro house, pop dance, pro..."
2,Capital Inicial,Rosas e Vinho Tinto,À Sua Maneira (De Música Ligeira),4Z0yuwHVJBROVZqFpTIr0d,"[brazilian rock, mpb, pop rock brasileiro, roc..."
3,Seu Jorge,"Musicas Para Churrasco, Vol. 1",Amiga Da Minha Mulher,0i1s9WcIu0PrUvHzALgofo,"[mpb, samba, samba-rock]"
4,Armin van Buuren,Not Giving Up On Love,Not Giving Up On Love - Radio Edit,0SfsnGyD8FpIN4U4WCkBZ5,"[dutch trance, edm, pop dance, progressive hou..."


In [161]:
genre_list_t=[]
genre = biblioteca['genre'].tolist()
for i in genre:
  if type (i) == list:
       for j in i:
          genre_list_t.append(j)
genre_list_t

['east coast hip hop',
 'hip hop',
 'pop rap',
 'rap',
 'dance pop',
 'edm',
 'electro house',
 'pop dance',
 'progressive electro house',
 'progressive house',
 'brazilian rock',
 'mpb',
 'pop rock brasileiro',
 'rock brasiliense',
 'rock nacional brasileiro',
 'mpb',
 'samba',
 'samba-rock',
 'dutch trance',
 'edm',
 'pop dance',
 'progressive house',
 'trance',
 'modern rock',
 'pop rap',
 'seattle hip hop',
 'uplifting trance',
 'neo mellow',
 'pop',
 'pop rock',
 'candy pop',
 'emo',
 'pixie',
 'pop emo',
 'pop punk',
 'dance pop',
 'edm',
 'electro house',
 'house',
 'pop',
 'progressive house',
 'uk dance',
 'album rock',
 'classic rock',
 'mellow gold',
 'rock',
 'hip hop',
 'miami hip hop',
 'pop rap',
 'rap',
 'southern hip hop',
 'trap',
 'dance pop',
 'edm',
 'electro house',
 'house',
 'pop',
 'progressive house',
 'uk dance',
 'detroit hip hop',
 'gangster rap',
 'hip hop',
 'pop rap',
 'rap',
 'permanent wave',
 'pop rock',
 'rock',
 'c-pop',
 'cantopop',
 'classic mando

In [162]:
genre_data_t={
    'genre': genre_list_t
}

In [163]:
genre_data_t=pd.DataFrame(genre_data_t)

In [164]:
genre_data_t.shape

(1272, 1)

In [165]:
genre_data_t['genre'].value_counts()

pop                     120
dance pop               106
post-teen pop            53
edm                      50
pop dance                44
                       ... 
latin alternative         1
trip hop                  1
hard rock brasileiro      1
pittsburgh rap            1
folk rock                 1
Name: genre, Length: 240, dtype: int64

# <font color= red> Spotify Natascha

## Pegando os dados de historico e concatenando num df só

In [166]:
historico1=pd.read_json('/content/drive/MyDrive/3_VAMO_AI/Dados_Natascha/Dados_Spotify_Descomprimido/MyData/StreamingHistory0.json')
historico1.head()

,endTime,artistName,trackName,msPlayed
0,2020-07-18 00:01,Metric,Gimme Sympathy - Acoustic,207040
1,2020-07-18 00:04,Metric,Gimme Sympathy - Acoustic,207040
2,2020-07-18 00:06,Mirah,Special Death,96810
3,2020-07-18 00:07,Beck,Ramona,45210
4,2020-07-18 00:11,The Bluetones,Sleazy Bed Track,19747


In [167]:
historico2=pd.read_json('/content/drive/MyDrive/3_VAMO_AI/Dados_Natascha/Dados_Spotify_Descomprimido/MyData/StreamingHistory1.json')
historico2.head()

,endTime,artistName,trackName,msPlayed
0,2020-11-02 00:10,Garbage,Stupid Girl,390
1,2020-11-02 00:10,Lady Gaga,Paparazzi,1070
2,2020-11-02 00:10,Ida Maria,Bad Karma,1450
3,2020-11-02 00:14,Gwen Stefani,The Sweet Escape,246466
4,2020-11-02 00:19,Joy Division,Dead Souls,295506


In [168]:
historico3=pd.read_json('/content/drive/MyDrive/3_VAMO_AI/Dados_Natascha/Dados_Spotify_Descomprimido/MyData/StreamingHistory2.json')
historico3.head()

,endTime,artistName,trackName,msPlayed
0,2021-02-16 17:36,La Roux,Bulletproof,205692
1,2021-02-16 17:39,Gino Paoli,Senza fine,167443
2,2021-02-16 17:45,Queen,Bohemian Rhapsody - Remastered 2011,354204
3,2021-02-16 17:48,Hole,Miss World,179912
4,2021-02-16 17:52,Marina Lima,Nosso Estranho Amor,225706


In [169]:
frames=(historico1,historico2,historico3)
hist_final=pd.concat(frames)
hist_final.shape

(25171, 4)

## Analisando o histórico
---



In [170]:
hist_final['trackName'].value_counts()

Circles                         194
I Ran (So Far Away)             190
Major Tom - Coming Home         139
Bad Guy                         137
I Was Made For Lovin' You       132
                               ... 
Crush                             1
Um Amor Puro                      1
Sem Nome, Mas Com Endereço        1
Pensando Bem                      1
You Learn - Live / Unplugged      1
Name: trackName, Length: 2000, dtype: int64

Tranformando em datetime para separar por ano, mês, dia e dia da semana

In [171]:
hist_final['endTime']=pd.to_datetime(hist_final['endTime'])

In [172]:
hist_final['year']=pd.DatetimeIndex(hist_final['endTime']).year
hist_final['month']=pd.DatetimeIndex(hist_final['endTime']).month
hist_final['day']=pd.DatetimeIndex(hist_final['endTime']).day
hist_final['day_name']=hist_final['endTime'].dt.day_name()

In [173]:
hist_final['part_of_day'] = hist_final['endTime'].dt.hour.apply(get_part_of_day)

In [174]:
hist_final["month"].replace(month, inplace=True)
hist_final.head()

,endTime,artistName,trackName,msPlayed,year,month,day,day_name,part_of_day
0,2020-07-18 00:01:00,Metric,Gimme Sympathy - Acoustic,207040,2020,July,18,Saturday,Late Night
1,2020-07-18 00:04:00,Metric,Gimme Sympathy - Acoustic,207040,2020,July,18,Saturday,Late Night
2,2020-07-18 00:06:00,Mirah,Special Death,96810,2020,July,18,Saturday,Late Night
3,2020-07-18 00:07:00,Beck,Ramona,45210,2020,July,18,Saturday,Late Night
4,2020-07-18 00:11:00,The Bluetones,Sleazy Bed Track,19747,2020,July,18,Saturday,Late Night


In [175]:
hist_final['day_name'].value_counts()

Thursday     4738
Tuesday      3914
Saturday     3719
Sunday       3654
Wednesday    3178
Friday       3145
Monday       2823
Name: day_name, dtype: int64

In [176]:
hist_final['trackName'].value_counts().head(10)

Circles                       194
I Ran (So Far Away)           190
Major Tom - Coming Home       139
Bad Guy                       137
I Was Made For Lovin' You     132
Dancing Queen                 124
I Touch Myself                123
Palavras No Corpo             118
Quando Bate Aquela Saudade    111
Mamma Mia                     108
Name: trackName, dtype: int64

In [177]:
hist_final['part_of_day'].value_counts()

Late Night       7824
Evening          5092
Night            4946
Noon             3549
Early Morning    1929
Morning          1831
Name: part_of_day, dtype: int64

## Abrindo o arquivo de biblioteca

In [178]:
with open('/content/drive/MyDrive/3_VAMO_AI/Dados_Natascha/Dados_Spotify_Descomprimido/MyData/YourLibrary.json', encoding="utf8") as f:
    biblioteca_nat=json.load(f)

type(biblioteca_nat)

dict

In [179]:
biblioteca_nat.keys()

dict_keys(['tracks', 'albums', 'shows', 'episodes', 'bannedTracks', 'artists', 'bannedArtists', 'other'])

Depois de transformar em dicionário, transformo cada índice num df

In [180]:
faixas=pd.json_normalize(biblioteca_nat['tracks'])
albuns=pd.json_normalize(biblioteca_nat['albums'])
podcasts=pd.json_normalize(biblioteca_nat['shows'])
artistas=pd.json_normalize(biblioteca_nat['artists'])

## Renomeando as colunas paraevitar duplicidade

In [181]:
faixas.head(1)

,artist,album,track,uri
0,Lykke Li,I Never Learn,Silver Line,spotify:track:3Thvm2A6Vcaia8AE7ycZkI


In [182]:
faixas.rename(columns={'artist':'faixas.artist','album':'faixas.album','track':'faixas.track','uri':'faixas.uri'},inplace=True)

In [183]:
albuns.head(1)

,artist,album,uri
0,Metric,Synthetica,spotify:album:1acDBFm5wQ58sJKv8VWoWo


In [184]:
albuns.rename(columns={'artist':'albuns.artist', 'album':'albuns.album', 'uri':'album.uri'},inplace=True)

In [185]:
podcasts.head(1)

,name,publisher,uri
0,Respondendo em Voz Alta,Laurinha Lero,spotify:show:0znjxREK2kgs1iL0i7W46V


In [186]:
podcasts.rename(columns={'name':'podcast.name','publisher':'podcast.publisher','uri':'podcast.uri'},inplace=True)

In [187]:
artistas.head(1)

,name,uri
0,ABBA,spotify:artist:0LcJLqbBmaGUft1e9Mm8HV


In [188]:
artistas.rename(columns={'name': 'artistas.name','uri':'artistas.uri'},inplace=True)

In [189]:
lista=[faixas, albuns, podcasts, episodios,outros]
biblioteca_nat=pd.concat(lista,axis=1)

In [190]:
biblioteca_nat.head()

,faixas.artist,faixas.album,faixas.track,faixas.uri,albuns.artist,albuns.album,album.uri,podcast.name,podcast.publisher,podcast.uri
0,Lykke Li,I Never Learn,Silver Line,spotify:track:3Thvm2A6Vcaia8AE7ycZkI,Metric,Synthetica,spotify:album:1acDBFm5wQ58sJKv8VWoWo,Respondendo em Voz Alta,Laurinha Lero,spotify:show:0znjxREK2kgs1iL0i7W46V
1,Los Hermanos,Ventura,Conversa de Botas Batidas,spotify:track:42yUlhx59y7G7EmMx5hKc0,Hole,Live Through This,spotify:album:2Rwf2nPYZQ9aIe4QXACTC7,falando sozinha,Laurinha Lero,spotify:show:2QkxibooVC8JMhXaeHLW4h
2,Lykke Li,Tonight,Tonight,spotify:track:5iP7DBxMKAZyanheMUNRFZ,The Killers,Battle Born,spotify:album:6P6DYLagXUhE5Qq2l79Ef1,NaN,NaN,NaN
3,David Bowie,The Man Who Sold The World,The Man Who Sold The World - 1999 Remastered V...,spotify:track:0jx7r5OF2yEXUyRArwlKzS,Tiê,Esmeraldas,spotify:album:6EtAuxIwZCEuUZEqn2DwII,NaN,NaN,NaN
4,La Roux,La Roux,I'm Not Your Toy,spotify:track:0SaJsihXgkqj0xEIbi8bnM,O Terno,Melhor do Que Parece,spotify:album:7nf6cMzbnj1BqhY5VBwT16,NaN,NaN,NaN


## Quais artistas eu mais tenho curtido

In [191]:
biblioteca_nat['faixas.artist'].value_counts()

The Killers       44
Lykke Li          38
P!nk              24
Joy Division      23
Hole              23
                  ..
Joe Cocker         1
Patrick Swayze     1
Arcade Fire        1
Gotye              1
Duran Duran        1
Name: faixas.artist, Length: 178, dtype: int64

## Spotipy

In [192]:
biblioteca_nat['artist_id']=biblioteca_nat.loc[:,'faixas.artist']

In [193]:
biblioteca_nat['artist_id']= biblioteca_nat['artist_id'].apply(get_artist_id)

In [194]:
biblioteca_nat.head(1)

,faixas.artist,faixas.album,faixas.track,faixas.uri,albuns.artist,albuns.album,album.uri,podcast.name,podcast.publisher,podcast.uri,artist_id
0,Lykke Li,I Never Learn,Silver Line,spotify:track:3Thvm2A6Vcaia8AE7ycZkI,Metric,Synthetica,spotify:album:1acDBFm5wQ58sJKv8VWoWo,Respondendo em Voz Alta,Laurinha Lero,spotify:show:0znjxREK2kgs1iL0i7W46V,6oBm8HB0yfrIc9IHbxs6in


In [195]:
biblioteca_nat['genre']=biblioteca_nat.loc[:,'faixas.artist']

In [196]:
biblioteca_nat['genre']=biblioteca_nat['genre'].apply(artist_genre)

In [197]:
biblioteca_nat.head(1)

,faixas.artist,faixas.album,faixas.track,faixas.uri,albuns.artist,albuns.album,album.uri,podcast.name,podcast.publisher,podcast.uri,artist_id,genre
0,Lykke Li,I Never Learn,Silver Line,spotify:track:3Thvm2A6Vcaia8AE7ycZkI,Metric,Synthetica,spotify:album:1acDBFm5wQ58sJKv8VWoWo,Respondendo em Voz Alta,Laurinha Lero,spotify:show:0znjxREK2kgs1iL0i7W46V,6oBm8HB0yfrIc9IHbxs6in,"[art pop, dance pop, electropop, pop, swedish ..."


Pegando cada genero listado no df e colocando numa unica lista

In [198]:
genre_list=[]
genre = biblioteca_nat['genre'].tolist()
for i in genre:
  for j in range(len(i)):
    genre_list.append(i[j])
genre_list

['art pop',
 'dance pop',
 'electropop',
 'pop',
 'swedish electropop',
 'swedish indie pop',
 'swedish pop',
 'cumbia sonidera',
 'grupera',
 'gruperas inmortales',
 'regional mexican',
 'tropical',
 'art pop',
 'dance pop',
 'electropop',
 'pop',
 'swedish electropop',
 'swedish indie pop',
 'swedish pop',
 'art rock',
 'classic rock',
 'glam rock',
 'permanent wave',
 'rock',
 'alternative dance',
 'dance pop',
 'electropop',
 'neo-synthpop',
 'new rave',
 'dance rock',
 'modern rock',
 'permanent wave',
 'rock',
 'disco',
 'europop',
 'italian disco',
 'vintage french electronic',
 'funk carioca',
 'pop lgbtq+ brasileira',
 'dance pop',
 'pop',
 'post-teen pop',
 'album rock',
 'art rock',
 'beatlesque',
 'classic rock',
 'mellow gold',
 'rock',
 'soft rock',
 'dance pop',
 'pop',
 'post-teen pop',
 'bossa nova',
 'mpb',
 'samba de roda',
 'tropicalia',
 'brazilian rock',
 'mpb',
 'rock brasiliense',
 'rock nacional brasileiro',
 'bossa nova',
 'mpb',
 'samba de roda',
 'tropicalia

In [199]:
genre_data_n={
    'genre': genre_list
}

In [200]:
genre_data_n=pd.DataFrame(genre_data_n)

In [201]:
genre_data_n.shape

(2888, 1)

# <Font color=red>Gmaps

## Carregando o json


In [202]:
def extract_activity(record):
    try:
        return record["activity"][0]["activity"][0]["type"]
    except:
        return None

In [203]:
with open ('/content/drive/MyDrive/3_VAMO_AI/01_Case_Ifood/Histórico de localização.json',encoding='utf-8') as f:
  gmaps=json.load(f)
gmaps

[{'accuracy': 24,
  'activity': [{'activity': [{'confidence': 38, 'type': 'STILL'},
     {'confidence': 38, 'type': 'UNKNOWN'},
     {'confidence': 23, 'type': 'IN_VEHICLE'}],
    'timestampMs': '1468850836455'}],
  'latitudeE7': -235145853,
  'longitudeE7': -463611635,
  'timestampMs': '1468850826205'},
 {'accuracy': 1500,
  'activity': [{'activity': [{'confidence': 44, 'type': 'IN_VEHICLE'},
     {'confidence': 27, 'type': 'STILL'},
     {'confidence': 27, 'type': 'UNKNOWN'},
     {'confidence': 2, 'type': 'ON_BICYCLE'}],
    'timestampMs': '1468850938727'}],
  'latitudeE7': -235101483,
  'longitudeE7': -463644180,
  'timestampMs': '1468850974589'},
 {'accuracy': 1500,
  'activity': [{'activity': [{'confidence': 58, 'type': 'UNKNOWN'},
     {'confidence': 29, 'type': 'IN_VEHICLE'},
     {'confidence': 12, 'type': 'STILL'},
     {'confidence': 2, 'type': 'ON_BICYCLE'}],
    'timestampMs': '1468851083948'},
   {'activity': [{'confidence': 48, 'type': 'STILL'},
     {'confidence': 31, '

In [204]:
type(gmaps)

list

Transformando num data frame

In [205]:
gmaps=pd.DataFrame(gmaps)

In [206]:
gmaps.head(1)


,timestampMs,latitudeE7,longitudeE7,accuracy,activity,altitude,verticalAccuracy,velocity,heading
0,1468850826205,-235145853,-463611635,24,"[{'timestampMs': '1468850836455', 'activity': ...",NaN,NaN,NaN,NaN


In [207]:
gmaps['latitudeE7']=gmaps['latitudeE7'].apply(lambda x: x/10000000)

In [208]:
gmaps['longitudeE7']=gmaps['longitudeE7'].apply(lambda x : x/10000000)

In [209]:
gmaps.head(5)

,timestampMs,latitudeE7,longitudeE7,accuracy,activity,altitude,verticalAccuracy,velocity,heading
0,1468850826205,-23.514585,-46.361164,24,"[{'timestampMs': '1468850836455', 'activity': ...",NaN,NaN,NaN,NaN
1,1468850974589,-23.510148,-46.364418,1500,"[{'timestampMs': '1468850938727', 'activity': ...",NaN,NaN,NaN,NaN
2,1468851089062,-23.510148,-46.364418,1500,"[{'timestampMs': '1468851083948', 'activity': ...",NaN,NaN,NaN,NaN
3,1468851104231,-23.514488,-46.361052,28,NaN,NaN,NaN,NaN,NaN
4,1468851129113,-23.514488,-46.361052,351,"[{'timestampMs': '1468851134225', 'activity': ...",NaN,NaN,NaN,NaN


## Transformando de ms para datetime

In [210]:
gmaps['timestampMs'] = pd.to_datetime(gmaps['timestampMs'], unit='ms')

separando ano, mes, dia e dia da semana

In [211]:
gmaps['year'] = pd.DatetimeIndex(gmaps['timestampMs']).year
gmaps['month'] = pd.DatetimeIndex(gmaps['timestampMs']).month
gmaps['day'] = pd.DatetimeIndex(gmaps['timestampMs']).day
gmaps['day_name'] = gmaps['timestampMs'].dt.day_name()
gmaps.head()

,timestampMs,latitudeE7,longitudeE7,accuracy,activity,altitude,verticalAccuracy,velocity,heading,year,month,day,day_name
0,2016-07-18 14:07:06.205,-23.514585,-46.361164,24,"[{'timestampMs': '1468850836455', 'activity': ...",NaN,NaN,NaN,NaN,2016,7,18,Monday
1,2016-07-18 14:09:34.589,-23.510148,-46.364418,1500,"[{'timestampMs': '1468850938727', 'activity': ...",NaN,NaN,NaN,NaN,2016,7,18,Monday
2,2016-07-18 14:11:29.062,-23.510148,-46.364418,1500,"[{'timestampMs': '1468851083948', 'activity': ...",NaN,NaN,NaN,NaN,2016,7,18,Monday
3,2016-07-18 14:11:44.231,-23.514488,-46.361052,28,NaN,NaN,NaN,NaN,NaN,2016,7,18,Monday
4,2016-07-18 14:12:09.113,-23.514488,-46.361052,351,"[{'timestampMs': '1468851134225', 'activity': ...",NaN,NaN,NaN,NaN,2016,7,18,Monday


Colocando as partes do dia de acordo com a faixa de horário

In [212]:
gmaps['part_of_day'] = gmaps['timestampMs'].dt.hour.apply(get_part_of_day)

mudando valores coluna mês para seus devidos nomes

In [213]:
gmaps["month"].replace(month, inplace=True)
gmaps.head(1)

,timestampMs,latitudeE7,longitudeE7,accuracy,activity,altitude,verticalAccuracy,velocity,heading,year,month,day,day_name,part_of_day
0,2016-07-18 14:07:06.205,-23.514585,-46.361164,24,"[{'timestampMs': '1468850836455', 'activity': ...",NaN,NaN,NaN,NaN,2016,July,18,Monday,Noon


## Dropando colunas que não usarei

In [214]:
gmaps.drop(labels=['altitude','verticalAccuracy','velocity','heading'],axis=1,inplace=True)

In [215]:
gmaps.head(1)

,timestampMs,latitudeE7,longitudeE7,accuracy,activity,year,month,day,day_name,part_of_day
0,2016-07-18 14:07:06.205,-23.514585,-46.361164,24,"[{'timestampMs': '1468850836455', 'activity': ...",2016,July,18,Monday,Noon


## Será que fazem sentindo esses dados soltos?

In [216]:
gmaps['day_name'].value_counts()

Friday       110683
Wednesday    106389
Thursday     105486
Tuesday      100882
Monday       100245
Saturday      99036
Sunday        86778
Name: day_name, dtype: int64

In [217]:
gmaps['month'].value_counts()

February     91113
January      74218
March        72906
November     62996
October      60665
December     57633
September    54792
August       52651
July         50219
April        48518
May          42106
June         41682
Name: month, dtype: int64

Locais mais visitados

In [218]:
gmaps['latitudeE7'].value_counts()

-23.514532    10615
-23.514493     9562
-23.510550     8016
-23.514516     7720
-23.510445     5605
              ...  
-23.553629        1
-23.544141        1
-23.535152        1
-23.520586        1
-22.218088        1
Name: latitudeE7, Length: 88573, dtype: int64

# <font color= red>Exportando os csvs

In [219]:
genre_data_t.to_csv('generos_thiago.csv')

In [220]:
genre_data_n.to_csv('generos_natascha.csv')

In [221]:
biblioteca.to_csv('biblioteca_thiago.csv')

In [222]:
biblioteca_nat.to_csv('biblioteca_natascha.csv')

In [223]:
historico_3.to_csv('historico_netflix_thiago.csv')

In [224]:
historico_visu.to_csv('historico_geral_netflix_thiago.csv')

In [225]:
historico_nat.to_csv('historico_netflix_natascha.csv')

In [226]:
ifood.to_csv('ifood.csv')

In [227]:
gmaps.to_csv('gmaps.csv')

In [228]:
sp_historico.to_csv('historico_thiago.csv')